# 测试tokenizer的训练用脚本
先data中下载相关数据集[tokenizer训练数据集](https://www.modelscope.cn/datasets/gongjy/minimind_dataset/resolve/master/pretrain_hq.jsonl)

In [1]:
#查看一下json数据结构
import json
data_path="../data/pretrain_hq.jsonl"
# 读取第一行并解析
with open(data_path, 'r', encoding='utf-8') as f:
    first_line = f.readline().strip()

data = json.loads(first_line)
print("第一条数据内容：")
print(json.dumps(data, ensure_ascii=False, indent=2))

第一条数据内容：
{
  "text": "<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的：\"如何成为一名成功的魔术师\" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>"
}


可以看到json表对应 text项是一个用户对话历史数据集
以<|im_start|>开始，<|im_end|>结束，代表着用户/model的content项


## 0.tokenizer基本流程
normalize->pre_tokenization->encoder->decoder<br>
Minimind采用了最基本的BPE方法，产生的是6400的size的token表，用来将文本encoder<br>
[文章1-LLM实践-tokenizer](https://zhuanlan.zhihu.com/p/739078635)<br>
[文章2-BPE,Sentencepiece等Tokenizer原理讲解](https://zhuanlan.zhihu.com/p/657047389)第6节开始的内容<br>

### 第一步：Normalization
这一步是：
- removing needless whitespace
- lowercasing
- removing accents<br>
也就是移除空格，大小写重置，removeing_accents就是café Gómez résumé变成cafe Gomez resume

### 第二步：Pre-tokenization
tokenizer 是不可以直接在原始的文本上训练的，需要做一些处理，比如这里的将句子切分成一个个词汇。这个环节叫做 Pre-tokenization。

## 初始化tokenizer和tokenizer的训练器

In [2]:
from tokenizers import (
    Tokenizer,
    models,
    trainers,
    pre_tokenizers,
    processors,
    decoders,
)
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

#定义特殊token
special_tokens = ["<|endoftext|>",
    "<|im_start|>",
    "<|im_end|>",
    "<pad>",
    "<mask>"]

# 定义训练器
trainer= trainers.BpeTrainer(
    vocab_size=6400,
    special_tokens=special_tokens,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    show_progress=True,
)

## 设计读取函数用于tokenizer训练读入数据

In [3]:
#采用的训练方法是train_from_iterator
#定义一个迭代器函数读取数据集
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            yield data['text']  # 假设每行数据有一个'text'字段
#测试一下读取的方法
data_path="../data/pretrain_hq.jsonl"
#打印两条数据
data_iter = read_data(data_path)
for _ in range(2):
    print(next(data_iter))
del data_iter

<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。
文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>
<|im_start|>根据输入的内容，编写一个类别标签。
这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要

## 开始训练
大概训练耗时33min左右

In [5]:
data_path="../data/pretrain_hq.jsonl"
texts= read_data(data_path)
#开始训练
tokenizer.train_from_iterator(texts, trainer=trainer)
#设置解码器
tokenizer.decoder= decoders.ByteLevel()
#验证一下decoder效果
assert tokenizer.token_to_id("<|endoftext|>") == 0
assert tokenizer.token_to_id("<|im_start|>") == 1
assert tokenizer.token_to_id("<|im_end|>") == 2
assert tokenizer.token_to_id("<pad>") == 3
assert tokenizer.token_to_id("<mask>") == 4
#保存tokenizer
import os
save_path = "../model"
os.makedirs(save_path, exist_ok=True)
tokenizer.save(os.path.join(save_path, "tokenizer.json"))
tokenizer.model.save(save_path)

['../model/vocab.json', '../model/merges.txt']

# 手动创建一份配置文件
[关于配置文件](https://blog.csdn.net/xiezhipu/article/details/145585777)<br>
[关于normalizer的配置说明](https://blog.csdn.net/weixin_49346755/article/details/126496833)<br>
[关于post_process的配置说明](https://blog.csdn.net/weixin_49346755/article/details/126499720)<br>
[chat_template的设计规则](https://www.guyuehome.com/detail?id=1888166611628642305)<br>
配置文件的作用主要是记录tokenizer的normalize,pre_process,post_process,template,special_token等tokenizer的关键参数

In [ ]:
# 手动创建配置文件
config = {
        "add_bos_token": False,
        "add_eos_token": False,
        "add_prefix_space": False,
        "added_tokens_decoder": {
            "0": {
                "content": "<|endoftext|>",
                "lstrip": False,
                "normalized": False,
                "rstrip": False,
                "single_word": False,
                "special": True
            },
            "1": {
                "content": "<|im_start|>",
                "lstrip": False,
                "normalized": False,
                "rstrip": False,
                "single_word": False,
                "special": True
            },
            "2": {
                "content": "<|im_end|>",
                "lstrip": False,
                "normalized": False,
                "rstrip": False,
                "single_word": False,
                "special": True
            },
            "3": {
                "content": "<pad>",
                "lstrip": False,
                "normalized": False,
                "rstrip": False,
                "single_word": False,
                "special": True
            },
            "4": {
                "content": "<mask>",
                "lstrip": False,
                "normalized": False,
                "rstrip": False,
                "single_word": False,
                "special": True
            }
        },
        "additional_special_tokens": [ "<pad>", "<mask>","<|多余的flag|>","<|zzy|>"],
        "bos_token": "<|im_start|>",
        "clean_up_tokenization_spaces": False,
        "eos_token": "<|im_end|>",
        "legacy": True,
        "model_max_length": 32768,
        "pad_token": "<|endoftext|>",
        "sp_model_kwargs": {},
        "spaces_between_special_tokens": False,
        "tokenizer_class": "PreTrainedTokenizerFast",
        "unk_token": "<|endoftext|>",
        "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{{ '<|im_start|>system\\n' + system_message + '<|im_end|>\\n' }}{% else %}{{ '<|im_start|>system\\nYou are a helpful assistant<|im_end|>\\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"
    }

save_path = "../model"
os.makedirs(save_path, exist_ok=True)
# 保存配置文件
with open(os.path.join(save_path, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


### 典型问题
config的added_tokens_decoder里<br>
我的"<|多余的flag|>","<|zzy|>"其实没有对应的encoder，所以会产生不对应的问题，这样的做法是错误的<br>
具体添加特殊标记的方法请参考模型的官方链接
[参考1：如何扩充词表](https://zhuanlan.zhihu.com/p/704346193#:~:text=%E7%AE%80%E5%8D%95%E6%9D%A5%E8%AF%B4%EF%BC%8C%E8%AF%BB%E5%85%A5%20tokenizer%20model%E4%B9%8B%E5%90%8E%EF%BC%8C%E8%B0%83%E7%94%A8%20tokenizer%20%E7%9A%84%20add_special_tokens%20%E6%96%B9%E6%B3%95%E7%BB%99%20tokenizer,model%20%E7%9A%84%20embedding%20size%EF%BC%8C%E9%80%9A%E8%BF%87%E8%B0%83%E7%94%A8%20model%20%E7%9A%84%20resize_token_embeddings%20%E6%96%B9%E6%B3%95%E6%9D%A5%E5%AE%9E%E7%8E%B0%E8%BF%99%E4%B8%80%E7%82%B9%E3%80%82)

#### template说明
chat_template结构：
```
{% if messages[0]['role'] == 'system' %}
  {% set system_message = messages[0]['content'] %}
  {{ '<|im_start|>system\n' + system_message + '<|im_end|>\n' }}
{% else %}
  {{ '<|im_start|>system\nYou are a helpful assistant<|im_end|>\n' }}
{% endif %}

{% for message in messages %}
  {% set content = message['content'] %}
  {% if message['role'] == 'user' %}
    {{ '<|im_start|>user\n' + content + '<|im_end|>\n<|im_start|>assistant\n' }}
  {% elif message['role'] == 'assistant' %}
    {{ content + '<|im_end|>' + '\n' }}
  {% endif %}
{% endfor %}
```

后续在读入我们的SFT的多轮对话数据集时候，就会用chat_template来转换数据集格式的。

#### 评估tokenizer的效果

In [10]:
from transformers import AutoTokenizer
# 评估tokenizer的效果
tokenizer_path="../model/"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True)

#后续SFT数据的格式
messages = [
        {"role": "system", "content": "你是一个优秀的聊天机器人，总是给我正确的回应！"},
        {"role": "user", "content": '你来自哪里？'},
        {"role": "assistant", "content": '我来自地球'}
    ]
new_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    ) #会把原始的dict转化为template格式的string
print(new_prompt)

#tokenizer理论词表长度
print("Tokenizer vocabulary size:", tokenizer.vocab_size)
#tokenizer实际长度
len_tokenizer = len(tokenizer)
print("Tokenizer actual size:", len_tokenizer)

#encoder测试
model_inputs=tokenizer(new_prompt)
print("length of model inputs:", len(model_inputs['input_ids']))
input_ids= model_inputs['input_ids']
print("Input IDs:", input_ids)
response= tokenizer.decode(input_ids, skip_special_tokens=False)
#比对response和new_prompt是否一致
print("Response matches new prompt:", response == new_prompt)

#打印tokenizer的特殊token
print("Tokenizer special tokens:")
print(tokenizer.special_tokens_map)

<|im_start|>system
你是一个优秀的聊天机器人，总是给我正确的回应！<|im_end|>
<|im_start|>user
你来自哪里？<|im_end|>
<|im_start|>assistant
我来自地球<|im_end|>

Tokenizer vocabulary size: 6400
Tokenizer actual size: 6402
length of model inputs: 46
Input IDs: [1, 87, 93, 307, 73, 81, 203, 397, 924, 5235, 3317, 2117, 265, 2603, 1132, 2599, 703, 472, 997, 2, 203, 1, 89, 87, 3709, 203, 397, 2722, 3016, 425, 2, 203, 1, 69, 87, 87, 77, 307, 3924, 88, 203, 301, 2722, 1284, 2, 203]
Response matches new prompt: True
Tokenizer special tokens:
{'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|多余的flag|>', '<|zzy|>']}


In [ ]:
#至此，tokenizer的训练和验证已经完成。可以看到，tokenizer能够正确地处理输入文本，并且生成的token与预期一致。接下来可以将这个tokenizer应用于模型训练或推理任务中。